In [ ]:
import jax
import jax.numpy as jnp
from jax import make_jaxpr

import genjax
from genjax import normal, pjax, pretty

pretty()

In [ ]:
# sampler: (Key, (x, y)) -> R
# sampler.batched(in_axes=(0, None)) -> sampler: (Key, (Tens_n(x), y)) -> Tens_n(R)
# sampler.batched(in_axes=(None, 0)) -> sampler: (Key, (x, Tens_n(y))) -> Tens_n(R)
# sampler.batched(in_axes=(0, 0)) -> sampler: (Key, (Tens_n(x), Tens_n(y))) -> Tens_n(R)
# sampler.batched(in_axes=(None, None), axis_size=n) -> sampler: (Key, (x, y)) -> Tens_n(R)

In [ ]:
# In PJAX (Jaxpr + sample_p) -- JAX (Python frontend + tracing -> JAXPR)
def prog(x):  # x : R -> P R
    v = normal.sample(1.0, 2.0)
    return x + v

**vmap semantics** `vmap : (A -> P T) -> (Vec A -> P Vec T)`

still "right" but it doesn't do the thing we expect?

In [ ]:
make_jaxpr(jax.vmap(prog, in_axes=(0,)))(jnp.ones(10))

In [ ]:
make_jaxpr(pjax.vmap(prog, in_axes=(0,)))(jnp.ones(10))

In [ ]:
@genjax.gen
def model(x):
    z = genjax.normal(x, 1.0) @ "z"
    return z


tr = jax.vmap(model.simulate, in_axes=(0,))((jnp.arange(0, 50, dtype=float),))
make_jaxpr(jax.vmap(lambda tr: tr.get_score()))(tr)

In [ ]:
@genjax.gen
def model(x):
    z = genjax.normal(x, 1.0) @ "z"
    return z


tr = jax.vmap(model.simulate, in_axes=(0,))((jnp.arange(0, 50, dtype=float),))
make_jaxpr(pjax.vmap(lambda tr: tr.get_score()))(tr)

In [ ]:
@genjax.gen
def model(x):
    z = genjax.normal(x, 1.0) @ "z"
    return z


tr = model.simulate((0.0,))
make_jaxpr(lambda tr: tr.get_score())(tr)

In [ ]:
from jax.core import ShapedArray

In [ ]:
jax.vmap(lambda v: ShapedArray(3, float))(jnp.ones(10))

In [ ]:
from jax.core import get_aval

In [ ]:
get_aval(jnp.ones(50))